# Testing

In [4]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from pyexplainer_pyexplainer import PyExplainer
import pickle
import os


cwd = os.getcwd()
parent_dir = os.path.dirname(cwd)
path_train = parent_dir + "/tests/pyexplainer_test_data/activemq-5.0.0.zip"
training_data = pd.read_csv(path_train, index_col = 'File')

dep = training_data.columns[-4]
selected_features = ["ADEV", "AvgCyclomaticModified", "AvgEssential", "AvgLineBlank", "AvgLineComment",
                     "CountClassBase", "CountClassCoupled", "CountClassDerived", "CountDeclClass",
                     "CountDeclClassMethod", "CountDeclClassVariable", "CountDeclInstanceVariable",
                     "CountDeclMethodDefault", "CountDeclMethodPrivate", "CountDeclMethodProtected",
                     "CountDeclMethodPublic", "CountInput_Mean", "CountInput_Min", "CountOutput_Min", "MAJOR_LINE",
                     "MaxInheritanceTree", "MaxNesting_Min", "MINOR_COMMIT", "OWN_COMMIT", "OWN_LINE",
                     "PercentLackOfCohesion", "RatioCommentToCode"]
all_cols = training_data.columns
for col in all_cols:
    if col not in selected_features:
        all_cols = all_cols.drop(col)
indep = all_cols
X_train = training_data.loc[:, indep]
y_train = training_data.loc[:, dep]

blackbox_model = RandomForestClassifier(max_depth=3, random_state=0)
blackbox_model.fit(X_train, y_train)

class_label = ['Clean', 'Defect']
pyExp = PyExplainer(X_train,
            y_train,
            indep,
            dep,
            blackbox_model,
            class_label=class_label)

path_test = parent_dir + "/tests/pyexplainer_test_data/activemq-5.1.0.zip"
sample_test_data = pd.read_csv(path_test, index_col = 'File')
X_test = sample_test_data.loc[:, indep]
y_test = sample_test_data.loc[:, dep]

sample_explain_index = 0
pyExp.X_explain = X_test.iloc[[sample_explain_index]]
pyExp.y_explain = y_test.iloc[[sample_explain_index]]

# Util functions for reading and writing data
def save_object(object_i, filename):
    with open(filename, 'wb') as file:
        pickle.dump(object_i, file)

def load_object(filename):
    with open(filename, 'rb') as file:
        object_o = pickle.load(file)
    return (object_o)

# load rule obj
if os.path.isfile('../tests/rule_objects/pyExplainer_obj.pyobject'):
    load_pyExp_rule_obj = load_object('../tests/rule_objects/pyExplainer_obj.pyobject')

In [5]:
%%time

d = []
for i in range(1000):
    X_explain = X_test.iloc[[i]]
    y_explain = y_test.iloc[[i]]
    pyExp.X_explain = X_test.iloc[[i]]
    pyExp.y_explain = y_test.iloc[[i]]
    rule = pyExp.explain(X_explain, y_explain, max_iter=10000)
    top_rule = pyExp.parse_top_rules(top_k_positive_rules=rule['top_k_positive_rules'],
                                     top_k_negative_rules=rule['top_k_negative_rules'])
    d.append(pyExp.generate_bullet_data(top_rule))


C:\Users\micha\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\micha\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result

KeyboardInterrupt: 